In [1]:
!pip install transformers

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=10, problem_type="multi_label_classification")

# New data for prediction
new_texts = ["This is a positive example.", "Another example but negative."]

# Preprocess input data
encoded_inputs = tokenizer(new_texts, padding=True, truncation=True, return_tensors="pt")
print(encoded_inputs)

# Make predictions
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    outputs = model(**encoded_inputs)

# The model outputs a tuple. The first element contains the logits for each class.
logits = outputs.logits

# You can use softmax to get probabilities if needed
probs = torch.softmax(logits, dim=-1)

# Assuming you want to get the predicted labels
predicted_labels = torch.argmax(logits, dim=-1)

print("Predicted Labels:", predicted_labels)


/mnt/c/Users/Sergi/Documents/BioTermCategorizer/.env_biotermcategorizer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3893, 2742, 1012,  102],
        [ 101, 2178, 2742, 2021, 4997, 1012,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0]])}
Predicted Labels: tensor([7, 7])


https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForSequenceClassification

In [3]:
from transformers import AutoConfig, AutoModelForSequenceClassification

# Download configuration from huggingface.co and cache.
config = AutoConfig.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_config(config)

In [4]:
from sklearn.metrics import classification_report
import numpy as np

labels = np.array(['A', 'B', 'C'])


y_true = np.array([1, 2, 0, 1, 2, 0])
y_pred = np.array([1, 2, 1, 1, 1, 0])
print(classification_report(y_true, y_pred, target_names=labels))

              precision    recall  f1-score   support

           A       1.00      0.50      0.67         2
           B       0.50      1.00      0.67         2
           C       1.00      0.50      0.67         2

    accuracy                           0.67         6
   macro avg       0.83      0.67      0.67         6
weighted avg       0.83      0.67      0.67         6



In [5]:
from sklearn.metrics import classification_report
import numpy as np

labels =['A', 'B', 'C']

y_true = np.array([[1, 0, 1],
                   [0, 1, 0],
                   [1, 1, 1]])
y_pred = np.array([[1, 0, 0],
                   [0, 1, 1],
                   [1, 1, 1]])

print(classification_report(y_true, y_pred, target_names=labels))

              precision    recall  f1-score   support

           A       1.00      1.00      1.00         2
           B       1.00      1.00      1.00         2
           C       0.50      0.50      0.50         2

   micro avg       0.83      0.83      0.83         6
   macro avg       0.83      0.83      0.83         6
weighted avg       0.83      0.83      0.83         6
 samples avg       0.83      0.83      0.78         6

